# Notebook for topic modeling 

# Imports

In [1]:
## load packages 
import pandas as pd
import re
import numpy as np

## nltk imports
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## sklearn imports
from sklearn.feature_extraction.text import CountVectorizer

## lda 
from gensim import corpora
import gensim

## viz
import pyLDAvis.gensim as gensimvis
import pyLDAvis

## print mult things
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## random
import random

# Load data

In [2]:
## full data
ab = pd.read_csv("../public_data/airbnb_text.zip")
ab.head()

## random sample to create ab small 
## removing null listings 
ab_small = ab.loc[~ab.name.isnull(),
           ['id', 'neighbourhood_group', 'price', 'name']].copy().rename(columns = {'price':
            'price_rawdata'}).sample(n = 1000, random_state = 478)

ab_small['name_lower'] = ab_small['name'].str.lower()
ab_small.head()


,id,name,name_upper,neighbourhood_group,price
0,2539,Clean & quiet apt home by the park,CLEAN & QUIET APT HOME BY THE PARK,Brooklyn,149
1,2595,Skylit Midtown Castle,SKYLIT MIDTOWN CASTLE,Manhattan,225
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,THE VILLAGE OF HARLEM....NEW YORK !,Manhattan,150
3,3831,Cozy Entire Floor of Brownstone,COZY ENTIRE FLOOR OF BROWNSTONE,Brooklyn,89
4,5022,Entire Apt: Spacious Studio/Loft by central park,ENTIRE APT: SPACIOUS STUDIO/LOFT BY CENTRAL PARK,Manhattan,80


,id,neighbourhood_group,price_rawdata,name,name_lower
39160,30553339,Brooklyn,50,Spacious Brooklyn Apartment,spacious brooklyn apartment
16681,13341820,Manhattan,239,Upscale 1 Bedroom Hell's Kitchen Apartment,upscale 1 bedroom hell's kitchen apartment
19815,15865251,Manhattan,60,Manhattan - Upper East Side Lovely Private Bed...,manhattan - upper east side lovely private bed...
43826,33901138,Brooklyn,81,Location! Room in Bklyn 10 minutes to Manhatta...,location! room in bklyn 10 minutes to manhatta...
26543,21121441,Manhattan,195,Bright 1 bd apartment next to west villiage,bright 1 bd apartment next to west villiage


# Activity

- Preprocess the texts
- Repeat the preprocessing steps and running of the topic model with preprocessed texts (can also play around with other parameters like n_topics)- what seems to produce useful topics?


If you get stuck on the preprocessing part, you can use below function and I show example of how to apply

## Preprocess and model estimation

In [3]:
def processtext(row, colname, stopword_list, min_token_length = 3):
    
    ## get string form of listing
    string_of_col = str(row[colname])
    try:
        ## remove stopwords 
        remove_stop = [word for word in wordpunct_tokenize(string_of_col)
                      if word not in stopword_list]
        processed_string = " ".join([porter.stem(i) 
                        for i in remove_stop if 
                        i.isalpha() and len(i) >= min_token_length])
        return(processed_string)
    except:
        processed_string = "" # to handle data errors where not actually text
        return(processed_string)

In [4]:
## more extensive stopwords list
list_stopwords = stopwords.words("english")
custom_words_toadd = ['apartment', 'new york', 'nyc',
                      'bronx', 'brooklyn', "room", "private", "apt",
                     'manhattan', 'queens', 
                      'staten island', "bedroom"]

list_stopwords_longer = list_stopwords + custom_words_toadd

## initialize stemmer
## initialize stemmer
porter = PorterStemmer()

In [5]:
## apply function row-wise (axis=1)
## and feed it, in addition to row, the 
## name of the lowercase text column
## and a list of stop words
ab_small['text_preprocess'] = ab_small.apply(processtext,
                             axis = 1,
                             args = ["name_lower", list_stopwords_longer])
ab_small.head()


,id,neighbourhood_group,price_rawdata,name,name_lower,text_preprocess
39160,30553339,Brooklyn,50,Spacious Brooklyn Apartment,spacious brooklyn apartment,spaciou
16681,13341820,Manhattan,239,Upscale 1 Bedroom Hell's Kitchen Apartment,upscale 1 bedroom hell's kitchen apartment,upscal hell kitchen
19815,15865251,Manhattan,60,Manhattan - Upper East Side Lovely Private Bed...,manhattan - upper east side lovely private bed...,upper east side love
43826,33901138,Brooklyn,81,Location! Room in Bklyn 10 minutes to Manhatta...,location! room in bklyn 10 minutes to manhatta...,locat bklyn minut
26543,21121441,Manhattan,195,Bright 1 bd apartment next to west villiage,bright 1 bd apartment next to west villiage,bright next west villiag


In [6]:
## filtering out any empty strings post preprocessing
ab_small = ab_small[ab_small.text_preprocess != ""].copy()
ab_small.shape

tokenized_text = [wordpunct_tokenize(one_text) for one_text in 
                                      ab_small.text_preprocess]

(988, 6)

In [7]:
## preprocess and estimate topicmod
### create dictionary
text_proc_dict = corpora.Dictionary(tokenized_text)
### filter dictionary- using 2% as bounds
text_proc_dict.filter_extremes(no_below = round(ab_small.shape[0]*0.05),
                             no_above = round(ab_small.shape[0]*0.95))

### create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict.doc2bow(one_text) 
                   for one_text in tokenized_text]

### estimate model
n_topics = 6
ldamod_proc = gensim.models.ldamodel.LdaModel(corpus_fromdict_proc, 
                                         num_topics = n_topics, id2word=text_proc_dict, 
                                         passes=6, alpha = 'auto',
                                        per_word_topics = True, random_state = 91988)


## Interpretation

In [8]:
### print topics and words
topics = ldamod_proc.print_topics(num_words = 10)
for topic in topics:
    print(topic)

(0, '0.869*"sunni" + 0.066*"spaciou" + 0.029*"park" + 0.015*"studio" + 0.006*"locat" + 0.005*"villag" + 0.004*"beauti" + 0.003*"cozi" + 0.003*"east"')
(1, '0.596*"spaciou" + 0.381*"locat" + 0.008*"beauti" + 0.005*"studio" + 0.003*"villag" + 0.003*"cozi" + 0.002*"sunni" + 0.001*"park" + 0.001*"east"')
(2, '0.533*"beauti" + 0.450*"park" + 0.004*"sunni" + 0.003*"locat" + 0.002*"cozi" + 0.002*"villag" + 0.002*"east" + 0.002*"studio" + 0.002*"spaciou"')
(3, '0.641*"studio" + 0.237*"park" + 0.045*"cozi" + 0.024*"east" + 0.021*"locat" + 0.011*"villag" + 0.008*"beauti" + 0.006*"spaciou" + 0.006*"sunni"')
(4, '0.844*"studio" + 0.109*"cozi" + 0.028*"beauti" + 0.006*"park" + 0.003*"spaciou" + 0.003*"villag" + 0.003*"locat" + 0.002*"east" + 0.002*"sunni"')
(5, '0.424*"cozi" + 0.302*"east" + 0.230*"villag" + 0.028*"sunni" + 0.007*"locat" + 0.005*"studio" + 0.002*"spaciou" + 0.001*"beauti" + 0.001*"park"')


In [9]:
### visualize
pyLDAvis.enable_notebook()
lda_display_proc = gensimvis.prepare(ldamod_proc, corpus_fromdict_proc, text_proc_dict)
pyLDAvis.display(lda_display_proc)

## if we move lambda to lower than 1, see stuff like
## topic 1 possibly reflecting east village
## topic 3 sunny near park

## Extra interpretation code relevant for problem set

What if we want to find which topics are associated with higher listing prices?

In [10]:
## get topic probabilities by doc and find mean listing by topic
### get document topics - list of list tuples
topic_probs_bydoc =[ldamod_proc.get_document_topics(item) for item in corpus_fromdict_proc]

## each document has a list containing topic, probability
## tuples- example w/ first document
one_list_tup = topic_probs_bydoc[0]
one_list_tup

## create a long for dataframe by flattening the list
topic_probs_bydoc_long = pd.DataFrame([t for lst in topic_probs_bydoc for t in lst],
                                     columns = ['topic', 'probability'])

## add id var- we're repeating each id in the original data k times
## for the number of topics
topic_probs_bydoc_long['doc_id'] = list(np.concatenate([[one_id] * n_topics for one_id in ab_small.id]).flat)

## pivot to wide format
topic_probs_bydoc_wide = pd.pivot_table(topic_probs_bydoc_long, index = ['doc_id'],
                        columns = ['topic']).reset_index().reset_index(drop = True)
topic_probs_bydoc_wide.columns = ['doc_id'] + ["topic_" + str(i) for i in np.arange(0, n_topics)]
topic_probs_bydoc_wide.head()


[(0, 0.072430894),
 (1, 0.5874715),
 (2, 0.08379004),
 (3, 0.06981018),
 (4, 0.07584252),
 (5, 0.11065491)]

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5
0,57754,0.143460,0.182438,0.166081,0.138371,0.150328,0.219322
1,62903,0.048400,0.061564,0.373293,0.046883,0.395855,0.074004
2,125053,0.143460,0.182438,0.166081,0.138371,0.150328,0.219322
3,174527,0.072377,0.092045,0.083790,0.069836,0.076009,0.605943
4,177421,0.072377,0.092063,0.579228,0.069810,0.075871,0.110651


In [11]:
## merge with original data using doc id
topic_wmeta = pd.merge(topic_probs_bydoc_wide,
                      ab_small,
                      left_on = 'doc_id',
                      right_on = 'id')

## create indicator for listing's top topic
topic_wmeta['toptopic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                    "topic_" in col]].idxmax(axis=1)
topic_wmeta.head()

## group by topic and find mean price
topic_wmeta.groupby('toptopic').agg({'price_rawdata': np.mean})

## group by borough and topic -- higher price for some also reflects
## diff borough composition
topic_wmeta.groupby(['toptopic', 'neighbourhood_group']).agg({'price_rawdata': np.mean})

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,id,neighbourhood_group,price_rawdata,name,name_lower,text_preprocess,toptopic
0,57754,0.143460,0.182438,0.166081,0.138371,0.150328,0.219322,57754,Manhattan,305,Stylish Large Gramercy Loft!,stylish large gramercy loft!,stylish larg gramerci loft,topic_5
1,62903,0.048400,0.061564,0.373293,0.046883,0.395855,0.074004,62903,Manhattan,205,Beautiful modern studio apartment in heart of NYC,beautiful modern studio apartment in heart of nyc,beauti modern studio heart,topic_4
2,125053,0.143460,0.182438,0.166081,0.138371,0.150328,0.219322,125053,Manhattan,395,⚡Quiet Gem w/roof deck on NY's Hottest Street⚡,⚡quiet gem w/roof deck on ny's hottest street⚡,quiet gem roof deck hottest street,topic_5
3,174527,0.072377,0.092045,0.083790,0.069836,0.076009,0.605943,174527,Brooklyn,150,Cozy private family home in Bushwick,cozy private family home in bushwick,cozi famili home bushwick,topic_5
4,177421,0.072377,0.092063,0.579228,0.069810,0.075871,0.110651,177421,Brooklyn,500,Brand New Beautiful Duplex Apartment with Garden,brand new beautiful duplex apartment with garden,brand new beauti duplex garden,topic_2


,price_rawdata
toptopic,
topic_0,121.540000
topic_1,135.513043
topic_2,159.911392
topic_3,145.500000
topic_4,131.345455
topic_5,154.396779


price_rawdata
toptopic neighbourhood_group               
topic_0  Brooklyn                125.757576
         Manhattan               125.642857
         Queens                   59.000000
         Staten Island            50.000000
topic_1  Bronx                   150.000000
         Brooklyn                116.040816
         Manhattan               169.745098
         Queens                   77.928571
topic_2  Bronx                    65.000000
         Brooklyn                146.194444
         Manhattan               190.500000
         Queens                  103.500000
topic_3  Brooklyn                111.500000
         Manhattan               162.500000
topic_4  Bronx                    80.000000
         Brooklyn                 93.153846
         Manhattan               160.818182
         Queens                   78.250000
topic_5  Bronx                    89.705882
         Brooklyn                118.405063
         Manhattan               201.616099
         Queens                  103.181818
         Staten Island            75.571429